In [2]:
import json
from qdrant_client import QdrantClient

# Buka file config.json dan baca API key
with open("../cred.json", "r") as file:
  config = json.load(file)

# Gunakan API key untuk koneksi ke Qdrant Cloud
client = QdrantClient(url=config["QDRANT_URL"], api_key=config["QDRANT_API_KEY"])

# Cek apakah koneksi berhasil
print(client.get_collections())

collections=[]


In [ ]:
import pandas as pd
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [ ]:
# Load data dari CSV
df = pd.read_csv("../data/desc_processed_penginapan_bukitvista.csv")  # Pastikan nama file sesuai
documents = df["processed_description"].astype(str).tolist()  # Ambil kolom deskripsi

# Tokenisasi tanpa preprocessing tambahan (karena sudah bersih)
tokenized_corpus = [word_tokenize(doc.lower()) for doc in documents]

# Inisialisasi BM25
bm25 = BM25Okapi(tokenized_corpus)

# Fungsi pencarian dengan BM25
def search_bm25(query, top_n=5):
  query_tokens = word_tokenize(query.lower())
  scores = bm25.get_scores(query_tokens)
  ranked_results = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
  return ranked_results[:top_n]

# Contoh pencarian
query = "sea view ubud"
results = search_bm25(query)

# Tampilkan hasil
for doc, score in results:
  print(f"Doc: {doc} | Score: {score}")

Doc: 4bedroom mediterranean luxury villa ubud rooftop view amazing pool amazing view jungle view pool view tropical villa bali vacation rental group friendly stay instagramable stay long stay trend property private private pool top trending experience perfect blend modern elegance tropical charm mediterranean luxury villa heart ubud feature private pool scenic jungle view rooftop beautifully design interior serene retreat ideal family group nestle close ubuds iconic rice terrace cultural landmark ultimate escape relaxation adventure experience perfect blend modern elegance tropical charm 4bedroom villa heart ubud feature private pool scenic rooftop view beautifully design interior serene retreat ideal family group nestle close ubuds iconic rice terrace cultural landmark ultimate escape relaxation adventure feature kingsize bed ac private pool fully equip kitchen rooftop breathtaking view highspeed wifi private oasis ubud 4bedroom villa ensure topnotch hospitality seamless service span 

In [ ]:
from qdrant_client.models import PointStruct

# Konversi data ke format Qdrant
points = [
  PointStruct(
    id=i,
    vector=row["vector"],  # Gunakan vektor hasil embedding
    payload={
      "author": row["author"],
      "address": row["address"],
      "processed_description": row["processed_description"],
      "numeric_features": [
        row["bedrooms"], row["rooms"], row["bathrooms"], row["garages"],
        row["max_guests"], row["months_since"], row["update_days_ago"], row["price_per_night"]
      ]
    }
  ) for i, row in df.iterrows()
]

# Insert ke Qdrant
client.upsert(collection_name="penginapan_bukitvista", points=points)

print("Data berhasil dimasukkan ke Qdrant!")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   author                 51 non-null     object 
 1   description            51 non-null     object 
 2   address                51 non-null     object 
 3   bedrooms               51 non-null     float64
 4   rooms                  51 non-null     float64
 5   bathrooms              51 non-null     float64
 6   garages                51 non-null     float64
 7   max_guests             51 non-null     float64
 8   months_since           51 non-null     float64
 9   update_days_ago        51 non-null     float64
 10  price_per_night        51 non-null     float64
 11  processed_description  51 non-null     object 
dtypes: float64(8), object(4)
memory usage: 4.9+ KB
